In [4]:
!pip install duckdb pandas sqlalchemy sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [5]:
import pandas as pd
df=pd.read_parquet("2015_flights.parquet")
df

,DEPARTURE_DELAY,ARRIVAL_DELAY,DISTANCE,SCHEDULED_DEPARTURE
0,-11.0,-22.0,1448,0.083333
1,-8.0,-9.0,2330,0.166667
2,-2.0,5.0,2296,0.333333
3,-5.0,-9.0,2342,0.333333
4,-1.0,-21.0,1448,0.416667
...,...,...,...,...
5819074,-4.0,-26.0,2611,23.983333
5819075,-4.0,-16.0,1617,23.983333
5819076,-9.0,-8.0,1598,23.983333
5819077,-6.0,-10.0,1189,23.983333


In [15]:
df.iloc[:,0:1].max()
df.iloc[:,0:1].min()
df.iloc[:,0:1].count()

DEPARTURE_DELAY    5732926
dtype: int64

In [6]:
%timeit -n 1 -r 1 df.to_parquet('2015_flights.saved.parquet')

478 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## DuckDB
It is faster than Pandas and takes lesser memory. 
Let's take the no of unique routes that were delayed by 1hr, 2hr etc. 
This takes longer on **Pandas** because
1. There are several delay buckets (1,2,3,...1988)
2. Unique count is a slow operation    

In [21]:
df.columns
delays = df.groupby('DEPARTURE_DELAY')['DISTANCE'].nunique()
delays[delays.index>0]

DEPARTURE_DELAY
1.0       1308
2.0       1300
3.0       1296
4.0       1289
5.0       1294
          ... 
1631.0       1
1649.0       1
1670.0       1
1878.0       1
1988.0       1
Name: DISTANCE, Length: 1162, dtype: int64

The above command took 0.4 seconds i.e., the pandas command. 
### Using DuckDB below.

In [33]:
import duckdb

delays_duckDB = duckdb.query('select DEPARTURE_DELAY, COUNT(DISTINCT DISTANCE) from "2015_flights.parquet" group by DEPARTURE_DELAY ORDER BY DEPARTURE_DELAY').to_df()
delays_duckDB[delays_duckDB['DEPARTURE_DELAY']>0]

,DEPARTURE_DELAY,count(DISTINCT DISTANCE)
55,1.0,1308
56,2.0,1300
57,3.0,1296
58,4.0,1289
59,5.0,1294
...,...,...
1212,1631.0,1
1213,1649.0,1
1214,1670.0,1
1215,1878.0,1


The above DUCKDB code ran in less than 0.2 seconds compared to pandas which took like around 0.8.
## DuckDB runs fantastic with Pandas.
We can use DuckDB whenever we want to and not entirely in the code while using pandas too.  

In [37]:
df1 = pd.read_parquet('2015_flights.parquet')
query1 = duckdb.query('select DEPARTURE_DELAY, COUNT(DISTINCT DISTANCE) from df1 group by DEPARTURE_DELAY ORDER BY DEPARTURE_DELAY').to_df()
query1

,DEPARTURE_DELAY,count(DISTINCT DISTANCE)
0,-82.0,1
1,-68.0,1
2,-61.0,1
3,-56.0,1
4,-55.0,1
...,...,...
1213,1649.0,1
1214,1670.0,1
1215,1878.0,1
1216,1988.0,1


In [49]:
df1 = pd.read_parquet('2015_flights.parquet')
df1 = duckdb.query('''
select
    CASE
        WHEN DISTANCE < 1000 THEN 'Short'
        WHEN DISTANCE >= 1000 AND DISTANCE < 2000 THEN 'Medium'
        WHEN DISTANCE > 2000 THEN 'Long'
    END AS Distance_Category,
    DEPARTURE_DELAY,
    ARRIVAL_DELAY DELAY
FROM df1''').to_df()
df1

,Distance_Category,DEPARTURE_DELAY,DELAY
0,Medium,-11.0,-22.0
1,Long,-8.0,-9.0
2,Long,-2.0,5.0
3,Long,-5.0,-9.0
4,Medium,-1.0,-21.0
...,...,...,...
5819074,Long,-4.0,-26.0
5819075,Medium,-4.0,-16.0
5819076,Medium,-9.0,-8.0
5819077,Medium,-6.0,-10.0


In [55]:
df1 = pd.read_parquet('2015_flights.parquet')
df1 = duckdb.query('''
select
    CASE
        WHEN DISTANCE < 1000 THEN 'Short'
        WHEN DISTANCE >= 1000 AND DISTANCE < 2000 THEN 'Medium'
        WHEN DISTANCE > 2000 THEN 'Long'
    END AS Distance_Category,
    DEPARTURE_DELAY,
    ARRIVAL_DELAY DELAY
FROM df1''').to_df()
query4 = duckdb.query('''
    SELECT Distance_Category, AVG(DEPARTURE_DELAY) AS Avg_Departure_Delay, AVG(DELAY) AS Avg_Arrival_Delay
    FROM df1
    WHERE DEPARTURE_DELAY > 0
    GROUP BY Distance_Category
''').to_df()
query4

,Distance_Category,Avg_Departure_Delay,Avg_Arrival_Delay
0,Long,29.886994,20.031525
1,Short,33.369848,29.740909
2,Medium,31.578835,24.225267


In [64]:
# Correlation between Departure and Arrival Delays.
df1 = pd.read_parquet('2015_flights.parquet')
df1 = duckdb.query('''
select
    CASE
        WHEN DISTANCE < 1000 THEN 'Short'
        WHEN DISTANCE >= 1000 AND DISTANCE < 2000 THEN 'Medium'
        WHEN DISTANCE > 2000 THEN 'Long'
    END AS Distance_Category,
    DEPARTURE_DELAY,
    ARRIVAL_DELAY DELAY
FROM df1''').to_df()
correlation = df1.groupby('Distance_Category').apply(lambda x: x[["DEPARTURE_DELAY","DELAY"]].corr().iloc[0,1]).reset_index(name = "Correlation")
correlation
# Using DuckDB to find average correlation
query5 = duckdb.query('''select AVG(Correlation) from correlation''').to_df()
query5


C:\Users\G Tejesh Mukunda\AppData\Local\Temp\ipykernel_25148\645531913.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  correlation = df1.groupby('Distance_Category').apply(lambda x: x[["DEPARTURE_DELAY","DELAY"]].corr().iloc[0,1]).reset_index(name = "Correlation")


,avg(Correlation)
0,0.936172


## Exercise 
Rank the ARRIVAL_DELAY for each DISTANCE. (Prioritize row with lower DEPARTURE_DELAY in case of a tie.)
Get rows with rank 1. (Solution will contain nearly 1300 rows.)